## 钢筋混凝土简支梁地震分析示例
*说明*：此脚本用于同济大学土木工程学院2024级研究生课程《桥梁智能设计与建造》阶段二 —— 有限元分析与智能评估

[![Teacher](https://img.shields.io/badge/授课教师-王晓伟-brightgreen)](https://faculty-civileng.tongji.edu.cn/wangxiaowei/zh_CN/index/34554)
[![Author](https://img.shields.io/badge/脚本作者-苟凌云-brightgreen)](https://github.com/ganansuan647)
[![Opstool开发及技术支持](https://img.shields.io/badge/Opstool开发及技术支持-闫业祥-brightgreen)](https://github.com/yexiang1992)

仅用于OpenSeesPy的学习交流。

参考资料请查看:

[OpenSeesPy文档](https://openseespydoc.readthedocs.io/en/latest/) | 
[Opstool文档](https://opstool.readthedocs.io/en/latest/) | 
[OpsCodeHint](https://github.com/yexiang1992/openseespy_code_hints)  → C:\ProgramData\anaconda3

In [ ]:
# 安装必要的模块（如已安装则跳过）
!pip install openseespy
!pip install opstool
!pip install numpy

In [1]:
# 导入必要的模块
import openseespy.opensees as ops  # OpenSeesPy主模块
from typing import Literal
import numpy as np

In [ ]:
print("开始钢筋混凝土简支梁地震分析示例")
ops.wipe()
# 单位规定：kN, m, s
# 创建ModelBuilder（三维模型，每个节点6个自由度）
ops.model('basic', '-ndm', 3, '-ndf', 6)

# ====== 参数定义 ====== #
L = 30  # 跨度长度(m)
H = 15  # 墩高(m)
nH, nL = 5, 10  # 高度和长度方向的分段数

g = 9.81  # 重力加速度(m/s^2)
m_girder = 50  # 每延米主梁质量(ton/m)
m_pier = 2.5*1*2  # 每延米墩柱质量(ton/m)
print("模型参数已定义！")

In [ ]:
# 创建墩的节点并添加质量
y = 0.0  # 所有点y坐标均为0
for i in range(2):
    # i=0 左墩，i=1 右墩
    x = i * L
    for j in range(nH+1):
        z = j * H / nH
        node_tag = (i+1)*100 + j+1
        ops.node(node_tag, x, y, z)
        # 墩柱质量分配(每段的质量均匀分给上下节点)
        M = m_pier*H    # 墩柱总质量
        m = M/nH/2 if (j == 0 or j == nH) else M/nH
        ops.mass(node_tag, m, m, m, 0.0, 0.0, 0.0)
    print(f"{'左' if i==0 else '右'}桥墩节点已创建！")

# 创建主梁的节点
for i in range(nL+1):
    x = i * L / nL
    z = H
    node_tag = 1000 + i+1
    ops.node(node_tag, x, y, z)
    # 主梁质量分配(每段的质量均匀分给左右节点)
    M = m_girder*L    # 主梁总质量
    m = M/nL/2 if (i == 0 or i == nL) else M/nL
    ops.mass(node_tag, m, m, m, 0.0, 0.0, 0.0)
print("主梁节点已创建！")

In [ ]:
# 固定墩柱底部
#  tag, DX, DY, DZ, RX, RY, RZ
ops.fix(101, 1, 1, 1, 1, 1, 1)
ops.fix(201, 1, 1, 1, 1, 1, 1)
print("桥墩底部已固定！")

In [ ]:
# 定义截面
# 定义非线性柱的材料
# ------------------------------------------
# 混凝土                          tag     f'c    ec0    f'cu    ecu
# 约束核心混凝土
ops.uniaxialMaterial("Concrete01", 1, -41000, -0.004, -34470, -0.014)
# 非约束保护层混凝土
ops.uniaxialMaterial("Concrete01", 2, -34470, -0.002, -25000, -0.006)
# 钢筋材料
fy, E, b = 400e3, 206.84e6, 0.01  # 屈服应力, 杨氏模量,硬化率
ops.uniaxialMaterial("Steel01", 3, fy, E, b)
# 墩柱尺寸
colWidth, colDepth = 1, 2
cover = 0.08
As = 0.02  # 钢筋面积

# 从参数派生的一些变量
y1, z1 = colDepth / 2.0, colWidth / 2.0
# 创建纤维截面
ops.section("Fiber", 991, "-GJ", 1e10)
# 创建混凝土核心纤维
ops.patch("rect", 1, 10, 1, cover - y1, cover - z1, y1 - cover, z1 - cover)
# 创建混凝土保护层纤维（顶部、底部、左侧、右侧）
ops.patch("rect", 2, 10, 1, -y1, z1 - cover, y1, z1)
ops.patch("rect", 2, 10, 1, -y1, -z1, y1, cover - z1)
ops.patch("rect", 2, 2, 1, -y1, cover - z1, cover - y1, z1 - cover)
ops.patch("rect", 2, 2, 1, y1 - cover, cover - z1, y1, z1 - cover)
# 创建钢筋纤维（右侧、中间、左侧）
ops.layer("straight", 3, 3, As, y1 - cover, z1 - cover, y1 - cover, cover - z1)
ops.layer("straight", 3, 2, As, 0.0, z1 - cover, 0.0, cover - z1)
ops.layer("straight", 3, 3, As, cover - y1, z1 - cover, cover - y1, cover - z1)
pier_sec = 1
ops.uniaxialMaterial("Elastic", 103, 1e10)
ops.section("Aggregator", pier_sec, *[103, "T"], "-section", 991)

print(f"墩柱截面已定义！tag={pier_sec}")

这里仅演示了利用基础方法创建纤维截面，详细如[patch](https://opensees.berkeley.edu/wiki/index.php?title=Patch_Command),[layer](https://opensees.berkeley.edu/wiki/index.php?title=Layer_Command),[section](https://opensees.berkeley.edu/wiki/index.php?title=Fiber_Section)命令的意义可以参考对应的Openseeswiki文档链接(见前)，这是因为[Openseespy文档的对应部分](https://openseespydoc.readthedocs.io/en/latest/src/fibersection.html)没有图片，不太直观

利用[Opstool](https://opstool.readthedocs.io/en/latest/src/notebooks/mod_vis_fibersec.html)也可以实现纤维截面的建模及可视化，这里不做演示，有兴趣可自行尝试

In [ ]:
# ====== 定义单元 ====== #
# 桥墩单元
# 定义单元之前先要定义几何变换
ops.geomTransf('Linear', 1, 0, 1, 0)
# 设置元素长度方向的积分点数量(dispBeamColumn需要)
NP = 5
# 使用Lobatto积分，id为2
ops.beamIntegration('Lobatto', 2, pier_sec, NP)
# 使用塑性梁柱单元创建桥墩
for i in range(2):
    for j in range(nH):
        node1 = (i+1)*100 + j+1
        node2 = (i+1)*100 + j+2
        ele_tag = (i+1)*100 + j+1
        # 倒数第二个参数是几何变换的tag，最后一个参数是积分点的tag
        ops.element('dispBeamColumn', ele_tag, node1, node2, 1, 2)
    print(f"{'左' if i==0 else '右'}桥墩单元已创建！")
        
# 定义梁元素的几何变换
ops.geomTransf('Linear', 2, 0, 1, 0)

# 创建弹性梁单元
for i in range(nL):
    ele_tag = 1000 + i + 1
    node1 = 1000 + i + 1
    node2 = node1 + 1
    #                                    tag, ndI,     ndJ,    A,     E,   Iz,   Iy,    G,    J, transfTag
    ops.element('elasticBeamColumn', ele_tag, node1, node2, 0.86, 210e6, 23.2, 2.32, 81e6, 3.13, 2)
print("主梁单元已创建！")

In [ ]:
# ====== 定义支座 ====== #
rigid_tag,free_tag = 9903,9904
ops.uniaxialMaterial('Elastic', rigid_tag, 1e7)  # 大刚度
ops.uniaxialMaterial('Elastic', free_tag, 10)  # 小刚度

def define_bearings(type:Literal['Rubber','Frame']):
    # 支座采用零长单元
    if type == 'Rubber':
        # 水平和竖向采用橡胶支座刚度，为橡胶支座
        # 以GBZJ500X650X110(CR)为例
        M_girder = m_girder*L  # 主梁质量(吨)
        At = 0.3136  # 支座面积(m^2)
        E = 462037.5  # 弹性模量 E=5.4GS^2(kPa)
        k = 4704  # 抗剪刚度(kN/m)
        fy = 0.2*M_girder/2*9.81  # 动摩擦力(kN)

        # 定义支座材料(根据规范计算)，假定有10个
        n_bear = 10
        ops.uniaxialMaterial('Elastic', 9901, n_bear*E*At)  # 竖向刚度
        ops.uniaxialMaterial('Steel01', 9902, fy, n_bear*k, 0.000001)  # 双线性本构
        ops.element('zeroLength',11, *[100+nH+1,1001],'-mat',*[9902,9902,9901,rigid_tag,free_tag,free_tag],'-dir',*[1,2,3,4,5,6])
        ops.element('zeroLength',21, *[200+nH+1,1000+nL+1],'-mat',*[9902,9902,9901,rigid_tag,free_tag,free_tag],'-dir',*[1,2,3,4,5,6])
        print("橡胶支座已定义！tag=[11,21]")
    elif type == 'Frame':   
        # 以下定义所有自由度均为大刚度，可以近似等效为框架（取消注释和上面的命令互换试试）
        ops.element('zeroLength',11, *[100+nH+1,1001],'-mat',*[rigid_tag,rigid_tag,rigid_tag,rigid_tag,rigid_tag,rigid_tag],'-dir',*[1,2,3,4,5,6])
        ops.element('zeroLength',21, *[200+nH+1,1000+nL+1],'-mat',*[rigid_tag,rigid_tag,rigid_tag,rigid_tag,rigid_tag,rigid_tag],'-dir',*[1,2,3,4,5,6])
        print("刚性支座连接已定义！tag=[11,21]")
    else:
        raise NotImplementedError(f"支座类型{type}未实现")

define_bearings(type='Rubber')

In [ ]:
# ====== 定义重力荷载 ====== #

# 创建带有线性时间序列的Plain荷载模式
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)

# 为每个点创建荷载
for node in ops.getNodeTags():
    # 读取节点质量,这里读取的是z向质量(dir=3)
    P = ops.nodeMass(node,3)*g    # 节点z向质量*重力加速度
    if P > 0:
        ops.load(node, 0.0, 0.0, -P, 0.0, 0.0, 0.0)
print("所有节点重力荷载已定义！")
print("模型定义完成!")

In [ ]:
# 定义完模型即可使用opstool协助进行可视化
import opstool as opst
# 获取模型数据
ModelData = opst.GetFEMdata(results_dir="opstool_output")
ModelData.get_model_data(save_file="ModelData.hdf5",print_model_info=True)
# 使用plotly进行模型可视化
opsvis = opst.OpsVisPlotly(point_size=2, line_width=3, colors_dict=None, theme="plotly",
                           color_map="jet", results_dir="opstool_output",on_notebook=True)
fig = opsvis.model_vis(input_file="ModelData.hdf5",
                 show_node_label=False,
                 show_ele_label=False,
                 show_local_crd=True,
                 show_fix_node=True,
                 show_constrain_dof=False,
                 label_size=8,
                 show_outline=True,
                 opacity=1.0)
fig.write_html("ModelData.html", auto_open=True)    # 在浏览器里自动打开
# fig.show()    # 如果在jupyternotebook里，请设置on_notebook=True
# 您可以更改函数参数以查看更改后的效果。

In [ ]:
# ====== 静力分析设置 ====== #
Nsteps = 10
ops.system('BandGeneral')   # 求解器类型，BandGeneral适用于带状矩阵，如梁柱结构
ops.constraints('Transformation')  # 约束处理方法，Transformation，适用于大多数问题
ops.numberer('RCM')  # 节点编号方法，RCM (Reverse Cuthill-McKee)算法，可以减少带宽
ops.test('NormDispIncr', 1.0e-12, 15, 3)  # 收敛测试:位移增量范数,容差1.0e-12,最大迭代数15
ops.algorithm('Newton')  # 解算法，Newton-Raphson法，适用于大多数非线性问题
ops.integrator('LoadControl', 1 / Nsteps) # Nsteps与步长的乘积一定要为1，代表施加一倍荷载，乘积为2代表施加两倍荷载
ops.analysis('Static')
print("静力分析设置完成")

如果对这些参数的作用和区别感兴趣，可以参考以下资料：

[system命令](https://mp.weixin.qq.com/s?__biz=MzI3MDQ3MDQ1MA%3D%3D&mid=2247484913&idx=1&sn=1ffc5bc5a489dd902cf1cfd77da2503c) | 
[numberer命令](https://mp.weixin.qq.com/s?__biz=MzI3MDQ3MDQ1MA%3D%3D&mid=2247484834&idx=1&sn=27f214abf1999a6e293056a6e775a46a) | 
[LoadControl步长](https://mp.weixin.qq.com/s?__biz=MzI3MDQ3MDQ1MA%3D%3D&mid=2247484820&idx=1&sn=251a494ebdd5e876d2607922ba663d79) | 


In [12]:
# start analysis loop
for i in range(Nsteps):
    ok = ops.analyze(1)
    ModelData.get_resp_step()
# save all responses data after loop
ModelData.save_resp_all(save_file="RespStepData-1.hdf5")

In [43]:
# 可视化(这里的x，y，z分别代表梁单元局部坐标系的x，y，z方向)
opsvis = opst.OpsVisPlotly(point_size=2, line_width=3, colors_dict=None, theme="plotly",
                           color_map="jet", results_dir="opstool_output",on_notebook=True)
fig = opsvis.frame_resp_vis(input_file="RespStepData-1.hdf5",
                      ele_tags=None,
                      slider=True,
                      response="Mz",
                      show_values=False,
                      alpha=1.0,
                      opacity=1)
fig.write_html("ModelDefo.html", auto_open=True)    # 在浏览器里自动打开
# fig.show()    # 如果在jupyternotebook里，请设置on_notebook=True

简支梁跨中弯矩和位移应该是多少呢？

跨中弯矩：$$M=q\cdot L^2/8 = 50ton\cdot 9.81\cdot 30^2/8 = 55181.25kN\cdot m$$
跨中位移：$$v=q\cdot L^4/(8EI) = 50ton\cdot 9.81\cdot 30^4/(8\cdot 210e6\cdot 2.32) = 0.012m$$

In [ ]:
# 也可以直接提取主梁挠度数据进行画图
import plotly.graph_objects as go
# 获取主梁节点位移
girder_disp = [ops.nodeDisp(1000+i+1, 3) for i in range(nL+1)]
# 创建x坐标
x = [i * L / nL for i in range(nL+1)]
# 创建图表
fig = go.Figure()
# 添加主梁挠度曲线
fig.add_trace(go.Scatter(x=x, y=girder_disp, mode='lines+markers', name='主梁挠度'))
# 设置图表布局
fig.update_layout(
    title='主梁挠度图',
    xaxis_title='跨度位置 (m)',
    yaxis_title='挠度 (m)',
    yaxis_zeroline=True,
    yaxis_zerolinewidth=2,
    yaxis_zerolinecolor='LightGray'
)
# 显示图表
fig.show()

In [ ]:
# 设置重力荷载为常量并重置域中的时间(这一步很重要，避免自重继续随时间增加)
ops.loadConst('-time', 0.0)
print("重力分析完成")

In [15]:
# 地震动分析
# 删除旧的分析
ops.wipeAnalysis()

# 修改分析设置以适应地震分析
ops.system('BandGeneral')
ops.constraints('Plain')
ops.test('NormDispIncr', 1.0e-12,  10)
ops.algorithm('Newton')
ops.numberer('RCM')
ops.integrator('Newmark',  0.5,  0.25)
ops.analysis('Transient')

In [ ]:
# 可以记录一下地震分析之前的结构周期
import math
numEigen = 3
eigenValues = ops.eigen(numEigen)
periods = [2*math.pi/math.sqrt(eigenValues[i]) for i in range(numEigen)]
print("地震分析开始时的周期(s):", periods)

# 利用前两阶周期计算瑞利阻尼因子
xi = 0.05 # 阻尼比0.05
omega1,omega2 = 2*math.pi/periods[0],2*math.pi/periods[1]
# 计算瑞丽阻尼系数 α 和 β
alpha = 2 * xi * (omega1 * omega2) / (omega1 + omega2)
beta = 2 * xi / (omega1 + omega2)
ops.rayleigh(alpha, 0.0, beta, 0.0)

In [48]:
# # 可视化前记得先获取可视化数据（取消注释即可使用）（如果上面运行过则不需要）
# import opstool as opst
# # 获取模型数据
# ModelData = opst.GetFEMdata(results_dir="opstool_output")
# ModelData.get_model_data(save_file="ModelData.hdf5",print_model_info=False)
# opsvis = opst.OpsVisPlotly(point_size=2, line_width=3, colors_dict=None, theme="plotly",
#                            color_map="jet", results_dir="opstool_output",on_notebook=True)

In [ ]:
# 到这里就可以进行模态分析了，还是用plotly（plotly对零长单元支持不太好）
ModelData.get_eigen_data(mode_tag=9, solver="-genBandArpack",
                         save_file='EigenData.hdf5')
fig = opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1,9], subplots=True,
                 alpha=1.0, show_outline=False,
                 show_origin=True, opacity=1.0,
                 show_face_line=True)
fig.show()

In [ ]:
# 如果plotly中的模态显示不全，可以尝试使用pyvista进行可视化
# 这里尝试使用pyvista进行模态可视化,注意不在notebook里运行请把on_notebook设为False
opsvis = opst.OpsVisPyvista(point_size=0, line_width=2,
                            colors_dict=None, theme="document",
                            color_map="coolwarm", on_notebook=True,
                            results_dir="opstool_output")

# 获取特征值数据
ModelData.get_eigen_data(mode_tag=9, solver="-genBandArpack",
                         save_file='EigenData.hdf5')

# 可视化特征值
opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1, 9], subplots=True,
                 alpha=1.0, show_outline=False,
                 show_origin=True, opacity=1.0,
                 show_face_line=False, save_fig='images/EigenVis2.svg')

In [51]:
# 如果上面用pyvista得到的结果是静态的，根据提示安装trame或其他包，并确保其与pyvista版本兼容
# !pip install trame==2.1.1

In [ ]:
# 读取地震动记录
from pathlib import Path
# 将earthquakerecord.txt文件放在同一目录下即可
filepath = Path.cwd() / 'earthquakerecord.txt'
print(f"地震记录文件路径：{filepath.absolute()}")
# 设置时间序列
nsteps,dt = 6062,0.01
ts_tag = 3
ops.timeSeries('Path', ts_tag, '-filePath', str(filepath), '-dt', dt, '-factor', g)
# 创建地震动荷载模式
axis = 1    # 地震动输入方向
ops.pattern('UniformExcitation', 2, axis, '-accel', ts_tag)

如何记录地震分析结果？

这个问题在进行动力分析之前就需要考虑。以下介绍三种方法：

1. 传统方法：创建recorder进行记录

    - 优点：可以直接写入到文件
    - 缺点：每步进行IO操作，速度慢

2. 推荐方法：利用Python提取响应

    - 优点：
        - 无需每步IO，速度快
        - 可以提取任意响应
        - 可以实时后处理
    - 缺点：几乎没有

3. 便捷方法：利用opstool记录所有结果

    - 优点：方便
    - 缺点：需要大量存储空间

In [ ]:
# 假设这里我们想记录墩顶位移，主梁中间节点加速度和主梁中间单元的加速度

# 先说最简单的第三种方法，只需要在记录的时候运行下面的记录函数
# ModelData.get_resp_step()
# 然后在计算完成后保存即可（这里基本上记录了所有信息）
# ModelData.save_resp_all(save_file="RespStepData-2.hdf5")

# 实际上，记录所有信息对于实际分析而言一般是不必要也不可能的，如果每步都记录很容易卡住，建议仅在调试时使用
# 一般需要自己定义记录的参数:

# 下面演示第一种方法，创建recorder
# 记录x方向主梁中间节点加速度
ops.recorder('Node', '-file', 'girder_acc.txt', '-timeSeries', ts_tag, '-time', '-dT', dt, '-node', 1000+int(nL/2)+1, '-dof', 1, 'accel')
# 记录x方向墩顶位移和转角
ops.recorder('Node', '-file', 'pier_disp.txt', '-time','-dT', dt, '-node', 100+nH+1, '-dof', *[1,5], 'disp')
# 记录桥墩剪力和弯矩,localForce代表局部坐标系下的力,globalForce代表全局坐标系下的力
ops.recorder('Element' ,'-file', 'pier_force','-time','-dT',dt,'-eleRange',101,101+nH,'localForce')

如果对于具体能记录哪些响应感兴趣，可以参考以下资料:
[recorder命令文档](https://openseespydoc.readthedocs.io/en/latest/src/recorder.html) | 
[公众号文章](https://mp.weixin.qq.com/s?__biz=MzI3MDQ3MDQ1MA%3D%3D&mid=2247484428&idx=1&sn=632bf8b1f71d72d493cab8ad4fbe2421)

下面演示第二种方法，每步提取响应，这里说说怎么提取响应

我们一般可以通过如下两种方式进行记录节点和单元的响应
1. 使用ops.nodeResponse(nodeTag, dof, responseID), 常用的如Disp=1, Vel=2, Accel=3, Reaction=6
2. 使用ops.eleResponse(eleTag, *args), 常用的如"deformation"，"axialForce"，["material", "stressStrain"]（方括号里的两个一起用）

其他命令参考[文档](https://openseespydoc.readthedocs.io/en/latest/src/outputcmds.html)

In [19]:
# 实际用法可以参考下面的例子，先定义一个提取函数
from collections import defaultdict

def save_results(results: defaultdict):
    Accel = ops.nodeAccel(1000+int(nL/2)+1, 1)  # 获取主梁中间节点的x向加速度
    results["Accel"].append(Accel)
    disp = ops.nodeDisp(100+nH+1, *[1,5])  # 获取左墩墩顶节点的x向位移和转角
    results["Disp"].append(disp)
    for etag in ops.getEleTags():  # 可以很方便的遍历单元号（如果需要）
        # 提单元内力
        resp = ops.eleResponse(etag, "localForce")
        results[f"AxisForce-Ele{etag}"].append(resp)
        # 提取单元变形
        resp = ops.eleResponse(etag, "deformation")
        results[f"AxisDefo-Ele{etag}"].append(resp)
        # 提取单元上材料应力应变
        resp = ops.eleResponse(etag, "material", "stressStrain")
        results[f"stressStrain-Ele{etag}"].append(resp)
        
    results["Time"].append(ops.getTime())  # 别忘了每步对应的时间
    
# 定义一个字典用于存储结果
RESULTS = defaultdict(lambda: [])

In [ ]:
# 接下来就可以进行地震分析了
# 目前比较推荐的方法是利用Opstool中的SmartAnalyze类进行地震分析
# 导入notebook中需要用到的clear_output函数，用于清除之前的输出
from IPython.display import clear_output
# SmartAnalyze可以控制自动更换分析方法，积分步长等等设置，非常方便，具体查看opstool文档，这里简单演示，不调整参数
analysis = opst.SmartAnalyze(analysis_type="Transient",)
segs = analysis.transient_split(nsteps)
for seg in segs:
    clear_output(wait=True)  # 清除之前的输出，不是分析必要的，只是为了看起来更美观
    analysis.TransientAnalyze(dt)   # 分析一个dt
    
    # 第一种记录方法，无需在此处进行任何操作
    
    # 第二种记录方法，每步利用提取函数提取响应
    save_results(RESULTS)
    
    # 第三种记录方法，使用opstool进行记录（这里每50步记录一次是为了演示速度）
    if seg%50==0:
        ModelData.get_resp_step()
        
# 保存地震分析结果(对应第三种方法)
ModelData.save_resp_all(save_file="RespStepData-2.hdf5")

In [ ]:
# 地震之后的结构周期发生了什么变化？
eigenValues = ops.eigen(numEigen)
periods = [2*math.pi/math.sqrt(eigenValues[i]) for i in range(numEigen)]
print("地震分析结束时的周期(s):", periods)

In [57]:
# 想不想知道地震过程中结构的受力情况？(简单看看Opstool记录的结果)
opsvis = opst.OpsVisPlotly(point_size=2, line_width=3, colors_dict=None, theme="plotly",
                           color_map="jet", results_dir="opstool_output",on_notebook=True)
fig = opsvis.frame_resp_vis(input_file="RespStepData-2.hdf5",
                      ele_tags=None,
                      slider=True,
                      response="Mz",
                      show_values=False,
                      alpha=1.0,
                      opacity=1)
fig.write_html("Modelearthquake.html", auto_open=True)    # 在浏览器里自动打开
# fig.show()

In [ ]:
# 第一种方法记录的形式是怎样？自行查看对应文件
# 怎样从文件中读取绘制？自行编程实现

# 第二种方法记录得到的字段有哪些？
print(RESULTS.keys())
# 简单两行代码即可转换为numpy数组，方便后续分析
for key, value in RESULTS.items():
    RESULTS[key] = np.array(value)

In [ ]:
# 绘制利用第二种方式记录到的结果
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 创建子图
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=("左墩墩顶节点的x向位移", "主梁中间节点的x向加速度", "支座单元的x向相对位移"))

# 绘制左墩墩顶节点的x向位移
fig.add_trace(
    go.Scatter(x=RESULTS["Time"], y=RESULTS["Disp"], mode='lines', name='墩顶位移'),
    row=1, col=1
)

# 绘制主梁中间节点的x向加速度
fig.add_trace(
    go.Scatter(x=RESULTS["Time"], y=RESULTS["Accel"], mode='lines', name='主梁加速度'),
    row=2, col=1
)

# 绘制支座单元的x向相对位移
bearing_deformation = RESULTS[f"AxisDefo-Ele{11}"]
bearing_x_defo = [defo[0] for defo in bearing_deformation]
fig.add_trace(
    go.Scatter(x=RESULTS["Time"], y=bearing_x_defo, mode='lines', name='支座相对位移'),
    row=3, col=1
)


# 更新布局
fig.update_layout(height=600, width=800, title_text="节点响应时程曲线")
fig.update_xaxes(title_text="时间 (s)", row=2, col=1)
fig.update_yaxes(title_text="位移 (m)", row=1, col=1)
fig.update_yaxes(title_text="加速度 (m/s²)", row=2, col=1)
fig.update_xaxes(title_text="时间 (s)", row=3, col=1)
fig.update_yaxes(title_text="位移 (m)", row=3, col=1)

# 显示图表
fig.show()


In [ ]:
# 对照一下我们的地震动输入？
import plotly.graph_objects as go

# 读取地震记录数据
filepath = 'earthquakerecord.txt'
earthquake_data = np.loadtxt(filepath)

# 创建时间数组
time = np.arange(0, len(earthquake_data) * dt, dt)

# 创建图表
fig = go.Figure()

# 添加地震记录曲线
fig.add_trace(go.Scatter(x=time, y=earthquake_data, mode='lines', name='地震记录'))

# 更新布局
fig.update_layout(
    title='输入地震动时程曲线',
    xaxis_title='时间 (s)',
    yaxis_title='加速度 (g)',
    width=800,
    height=400
)

# 显示图表
fig.show()

print("地震记录的持续时间：{:.2f}秒".format(time[-1]))
print("地震记录的最大加速度：{:.4f}g".format(np.max(np.abs(earthquake_data))))
